In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100,
                                         shuffle=False, num_workers=2)

100%|██████████| 170M/170M [00:06<00:00, 26.0MB/s]


Build the model architecture

In [2]:
class DeeperCNN(nn.Module):
    def __init__(self):
        super(DeeperCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)  # Adding an Extra Convolutional Layer

        self.fc1 = nn.Linear(64 * 4 * 4, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x))) # ReLu Activation Function
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 64 * 4 * 4)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = DeeperCNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9) # Parameters

Train and Evaluate the model

In [4]:
def train(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    for inputs, targets in loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()*inputs.size(0)
    epoch_loss = running_loss/len(loader.dataset)
    return epoch_loss

def test(model, loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    acc = 100.*correct/total
    return acc

num_epochs = 200
for epoch in range(num_epochs):
    train_loss = train(model, trainloader, optimizer, criterion, device)
    test_acc = test(model, testloader, device)
    print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {train_loss:.4f}, Test Acc: {test_acc:.2f}%")

Epoch [1/200] Loss: 2.2305, Test Acc: 25.31%
Epoch [2/200] Loss: 2.0213, Test Acc: 33.07%
Epoch [3/200] Loss: 1.8331, Test Acc: 37.27%
Epoch [4/200] Loss: 1.6949, Test Acc: 42.22%
Epoch [5/200] Loss: 1.6061, Test Acc: 44.93%
Epoch [6/200] Loss: 1.5415, Test Acc: 47.12%
Epoch [7/200] Loss: 1.4887, Test Acc: 49.44%
Epoch [8/200] Loss: 1.4437, Test Acc: 51.79%
Epoch [9/200] Loss: 1.4116, Test Acc: 53.42%
Epoch [10/200] Loss: 1.3727, Test Acc: 53.73%
Epoch [11/200] Loss: 1.3390, Test Acc: 55.35%
Epoch [12/200] Loss: 1.2982, Test Acc: 56.69%
Epoch [13/200] Loss: 1.2684, Test Acc: 58.31%
Epoch [14/200] Loss: 1.2423, Test Acc: 59.66%
Epoch [15/200] Loss: 1.2149, Test Acc: 59.52%
Epoch [16/200] Loss: 1.1863, Test Acc: 61.21%
Epoch [17/200] Loss: 1.1582, Test Acc: 61.45%
Epoch [18/200] Loss: 1.1387, Test Acc: 63.39%
Epoch [19/200] Loss: 1.1145, Test Acc: 64.30%
Epoch [20/200] Loss: 1.0957, Test Acc: 64.48%
Epoch [21/200] Loss: 1.0817, Test Acc: 62.83%
Epoch [22/200] Loss: 1.0645, Test Acc: 64.1